In [2]:
import deepcalo as dpcal
import json
import keras as ks
import tensorflow as tf
import sys
import onnxruntime as ort
import os
import tensorflow as tf
import tf2onnx

In [4]:
print(dpcal.__version__)
print(sys.version)
print(ort.__version__)
print(tf2onnx.__version__)

0.2.3
3.8.6 (v3.8.6:db455296be, Sep 23 2020, 13:31:39) 
[Clang 6.0 (clang-600.0.57)]
1.7.0
1.8.5


In [14]:
!ls ../Downloads/Zee_mc_1000_epochs_3_8_5

combine_model.0008-31.3847.hdf5 model.0046-2.9464.json
dataparams.pkl                  model.h5
dataparams.txt                  weights.0046-2.9464.hdf5


In [15]:
model_path = '../Downloads/Zee_mc_1000_epochs_3_8_5/model.0046-2.9464.json'
print(model_path)

../Downloads/Zee_mc_1000_epochs_3_8_5/model.0046-2.9464.json


In [16]:
with open(model_path, 'r') as model_json:
    arch = json.load(model_json)
    model = ks.models.model_from_json(arch, custom_objects={'FiLM': dpcal.layers.FiLM()})

In [17]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
em_barrel (InputLayer)          [(None, 56, 11, 4)]  0                                            
__________________________________________________________________________________________________
scalars (InputLayer)            [(None, 16)]         0                                            
__________________________________________________________________________________________________
up_sampling2d (UpSampling2D)    (None, 56, 55, 4)    0           em_barrel[0][0]                  
__________________________________________________________________________________________________
scalar_net (Functional)         (None, 256)          5120        scalars[0][0]                    
____________________________________________________________________________________________

In [19]:
model.load_weights("../Downloads/Zee_mc_1000_epochs_3_8_5/weights.0046-2.9464.hdf5")

In [20]:
spec = (

    tf.TensorSpec((None,  5), tf.float32, name="event_info"),

    tf.TensorSpec((None,  10, 13, 1), tf.float32, name="tracks"),

    tf.TensorSpec((None,  56, 11, 4), tf.float32, name="em_barrel"),

    tf.TensorSpec((None,  1), tf.float32, name="multiply_output_with"),

    tf.TensorSpec((None,  16), tf.float32, name="scalars"),

)

## Convert keras model to Onnx

In [21]:
model_proto, _ = tf2onnx.convert.from_keras(

    model, input_signature=spec, opset=13, output_path="../Downloads/model.0046_2.9464.onnx")

Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`


## Validating the Onnx model

In [22]:
sess_ort = ort.InferenceSession("../Downloads/model.0046_2.9464.onnx")

In [25]:
for i in range(len(sess_ort.get_inputs())):
    print(sess_ort.get_inputs()[i].name,":",sess_ort.get_inputs()[i].shape)

event_info : ['unk__224', 5]
tracks : ['unk__225', 10, 13, 1]
em_barrel : ['unk__226', 56, 11, 4]
multiply_output_with : ['unk__227', 1]
scalars : ['unk__228', 16]


In [27]:
output_name = sess_ort.get_outputs()[0].name
print("output_name", output_name)
output_shape = sess_ort.get_outputs()[0].shape
print("output shape", output_shape)
output_type = sess_ort.get_outputs()[0].type
print("output type", output_type)

output_name multiply
output shape ['unk__229', 1]
output type tensor(float)
